Step 1- Install YOLO + Kaggle


In [ ]:
!pip install ultralytics kaggle -q

Step 2- Upload Kaggle API Key

In [ ]:
from google.colab import files
files.upload()   # upload kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Step 3- Download Dataset from Kaggle

In [ ]:
!kaggle datasets download -d ashfakyeafi/road-vehicle-images-dataset
!unzip road-vehicle-images-dataset.zip -d vehicle_dataset

Step 4- Prepare Dataset for YOLO

In [ ]:
import os

base_path = "/content/vehicle_dataset"
for root, dirs, files in os.walk(base_path):
    print(root, len(files))

Step 5- Train YOLOv8m

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

model.train(
    data="/content/vehicle_dataset/trafic_data/data_1.yaml",
    epochs=20,
    imgsz=460,
    batch=4
)

Step 6- Validate the Model

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/runs/detect/train/weights/final.pt")  # path to weights

metrics = model.val()
metrics

Upload your own test image:

Step 8- Export Best Weights

In [ ]:
from google.colab import files

files.download('/content/runs/detect/train/weights/final.pt')

Step 9- Optional: Add Gradio Interface

In [ ]:
import gradio as gr
from ultralytics import YOLO

model = YOLO("yolov8m.pt")

def detect_image(img):
    results = model(img)

    # Get annotated image
    annotated_img = results[0].plot()

    # Extract list of detected object names
    classes = results[0].names  # class_id to label map
    detections = results[0].boxes.cls.tolist()  # list of class IDs
    detected_objects = list(set([classes[int(cls_id)] for cls_id in detections]))

    return annotated_img, ", ".join(detected_objects) if detected_objects else "No objects detected"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 YOLOv8 Image Detection")
    gr.Markdown("Upload an image to detect objects.")

    with gr.Tab("Image Detection"):
        with gr.Row():
            image_input = gr.Image(type="pil", label="Upload Image")
            image_output = gr.Image(type="pil", label="Detected Image")

        with gr.Row():
            object_list = gr.Textbox(label="Detected Objects", interactive=False)

        with gr.Row():
            submit_btn = gr.Button("Submit")

        submit_btn.click(fn=detect_image, inputs=image_input, outputs=[image_output, object_list])

demo.launch(share=True)